# Script to alter the til tracking numpy arrays if an error has occurred

In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
#from cartopy import crs
import collections
from glob import glob

In [2]:
import PIL
print('PIL',PIL.__version__)
from PIL import Image
Image.MAX_IMAGE_PIXELS = 999999999999

PIL 8.4.0


In [38]:
# Specify the parameters to download images
tiles_remaining = "tile_name_tile_url_remaining_expanded.npy"
tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_errors = "tile_name_tile_url_error_downloading.npy"

tiles_remaining_array = np.load(tiles_remaining)
tiles_labeled_array = np.load(tiles_labeled)
tiles_errors_array = np.load(tiles_errors)
tiles_errors_array.shape

parent_directory = "C:\chips_allocation"
annotation_directory = "Check"
number_of_tiles = 25

In [45]:
tiles_labeled

'tile_name_tile_url_labeled.npy'

In [4]:
# reshape errors array into 2d array
#tiles_errors_array = np.reshape(tiles_errors_array, (int(tiles_errors_array.size/2),2))
#np.save('tile_name_tile_url_error_downloading.npy', tiles_errors_array)

## Double check that remaining images have not been labeled 

In [26]:
for i in range(len(tiles_remaining_array)):
    if tiles_remaining_array[i] in tiles_labeled_array:
        print(tiles_remaining_array[i])

## Confirm that all Tiles in tile error raise an error 


In [117]:
temp_error = tiles_errors_array[73:]
np.save('temp_error.npy', temp_error)
temp_error= 'temp_error.npy'
temp_error_array = np.load('temp_error.npy')

dist = ap.annotator(annotation_directory) #create the processing class
dist.state_dcc_directory(parent_directory)
dist.number_of_tiles(number_of_tiles)
dist.get_tile_urls(temp_error)
dist.make_subdirectories()
dist.download_images()
#dist.tile_rename()

Unlabeled Tiles (0, 2)


IndexError: index 0 is out of bounds for axis 0 with size 0

### Add tiles that do not raise an error to remaining array

In [43]:
#tiles_remaining_array.shape
working_tiles = [3, 10, 11, 12, 13, 43]
#tiles_remaining_array = np.concatenate((tiles_remaining_array, tiles_errors_array[working_tiles]))
#Save updated remaining tiles array
#np.save(tiles_remaining, tiles_remaining_array)

#save updated errors array
#tiles_errors_array = np.delete(tiles_errors_array,[3, 10, 11, 12, 13, 43], axis=0).shape
#np.save(tiles_errors, tiles_errors_array)

Load the annotator class

## Find any tiles with errors

In [8]:
dist = ap.annotator(annotation_directory) #create the processing class
dist.state_dcc_directory(parent_directory)
dist.number_of_tiles(number_of_tiles)
dist.get_tile_urls(tiles_remaining)
dist.make_subdirectories()
dist.download_images()
#dist.tile_rename()

Unlabeled Tiles (25, 2)
0
Bypassing download of already-downloaded file m_3209934_nw_14_060_20181028.tif
1
Bypassing download of already-downloaded file m_3309753_se_14_060_20181120.tif
2
Bypassing download of already-downloaded file m_2609838_sw_14_060_20190105_20190517.tif
3
Bypassing download of already-downloaded file m_3310236_ne_13_060_20180928.tif
4
Bypassing download of already-downloaded file m_3108809_ne_16_060_20181028_20190208.tif
5
Bypassing download of already-downloaded file m_3108916_ne_16_060_20181028_20190208.tif
6
Bypassing download of already-downloaded file m_3108920_nw_16_060_20181103_20190208.tif
7
Bypassing download of already-downloaded file m_4709708_nw_14_060_20180901_20190211.tif
8
Bypassing download of already-downloaded file m_4009806_nw_14_060_20180710_20181210.tif
9
Bypassing download of already-downloaded file m_4710315_nw_13_060_20180901_20190222.tif
10
Bypassing download of already-downloaded file m_3508301_se_17_060_20181021.tif
11
Bypassing download

['C:\\chips_allocation/Check\\tiles\\https_naipeuwest.blob.core.windows.net_naip_v002_tx_2018_tx_060cm_2018_32099_m_3209934_nw_14_060_20181028.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipeuwest.blob.core.windows.net_naip_v002_tx_2018_tx_060cm_2018_33097_m_3309753_se_14_060_20181120.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipeuwest.blob.core.windows.net_naip_v002_tx_2018_tx_060cm_2018_26098_m_2609838_sw_14_060_20190105_20190517.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipeuwest.blob.core.windows.net_naip_v002_tx_2018_tx_060cm_2018_33102_m_3310236_ne_13_060_20180928.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipeuwest.blob.core.windows.net_naip_v002_ms_2018_ms_060cm_2018_31088_m_3108809_ne_16_060_20181028_20190208.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipeuwest.blob.core.windows.net_naip_v002_ms_2018_ms_060cm_2018_31089_m_3108916_ne_16_060_20181028_20190208.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipeuwest.blob.core.windows.net_nai

## Fix Inproperly recorded tiles tiles that have been allocated but not yet labeled, and marked as remaining to be allocated

In [35]:
#Determine the tiles that have been allocated but not yet labeled
tiles_allocated = []
allocated_sets = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk/uncomplete_sets"

for sub_directory in ap.list_of_sub_directories(allocated_sets):
    tiles_allocated += glob(sub_directory+'/tiles/*.tif') #

#Identify what tiles have been mistakenly recorded as needing to be labeled
misrecorded_already_allocated_tiles = []
misrecorded_already_allocated_tiles_index = []
temp_remaining_array = tiles_remaining_array

for tile in tiles_allocated:
    tile_id_allocated = tile.rsplit("\\")[1].split("_",4)[4] #tile ids from allocated 
   
    if tile_id_allocated in tiles_remaining_array[:,0]:
        misrecorded_already_allocated_tiles.append(tiles_remaining_array[i])
        temp_remaining_array = temp_remaining_array[(temp_remaining_array[:,0] != tile_id_allocated),:] 

#Update the array tracking the remaining arrays 
tiles_remaining_array = temp_remaining_array
#np.save(tiles_remaining, tiles_remaining_array)

#Update the array tracking labeled arrays 
misrecorded_already_allocated_tiles = np.array(misrecorded_already_allocated_tiles)
misrecorded_already_allocated_tiles
#tiles_labeled_array = np.concatenate((tiles_labeled_array, misrecorded_already_allocated_tiles))
#np.save(tiles_labeled, tiles_labeled_array)

array([['m_2909601_ne_14_060_20181116_20190426.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2018/tx_060cm_2018/29096/m_2909601_ne_14_060_20181116_20190426.tif'],
       ['m_2909601_ne_14_060_20181116_20190426.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2018/tx_060cm_2018/29096/m_2909601_ne_14_060_20181116_20190426.tif'],
       ['m_2909601_ne_14_060_20181116_20190426.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2018/tx_060cm_2018/29096/m_2909601_ne_14_060_20181116_20190426.tif'],
       ['m_2909601_ne_14_060_20181116_20190426.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2018/tx_060cm_2018/29096/m_2909601_ne_14_060_20181116_20190426.tif'],
       ['m_2909601_ne_14_060_20181116_20190426.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/tx/2018/tx_060cm_2018/29096/m_2909601_ne_14_060_20181116_20190426.tif']],
      dtype='<U120')

# Code to Fix arrays 

Fix arrays when a tile cannot be found (HTTP Error 404: The specified blob does not exist.)

In [16]:
"""
row_to_remove = 21

#dist.tile_name_tile_url_tiles_for_annotators[row_to_remove]

error_downloading = np.concatenate((np.load('tile_name_tile_url_error_downloading.npy'),
                                    dist.tile_name_tile_url_tiles_for_annotators[row_to_remove]))
np.save('tile_name_tile_url_error_downloading.npy', error_downloading)

#Remove the rows with errors
print(np.load('tile_name_tile_url_remaining_expanded.npy').shape)
np.save('tile_name_tile_url_remaining_expanded',np.delete(np.load(tiles_remaining), row_to_remove, axis = 0))
print(np.load('tile_name_tile_url_remaining_expanded.npy').shape)
row_to_remove = np.nan
"""

(47, 2)
(46, 2)


In [ ]:
#recover the unlabeled images from the dist
## np.save('tile_name_tile_url_remaining_expanded',dist.tile_name_tile_url_unlabeled)
#dist.tile_name_tile_url_tiles_for_annotators

In [ ]:
#dist.track_tile_annotations(tiles_labeled)
#np.save('tile_name_tile_url_remaining_expanded', dist.tile_name_tile_url_remaining)
#np.save('tile_name_tile_url_labeled', dist.tile_name_tile_url_labeled)v

Fix arrays when tiles are not properly allocated

In [6]:
"""
N = #specify the number of tiles that were not properly allocated
#load the array
remaining = np.load('tile_name_tile_url_remaining_expanded.npy')
labeled = np.load("tile_name_tile_url_labeled.npy")
#Save the ammended arrays
#np.save('tile_name_tile_url_labeled',labeled[:-N])
#np.save('tile_name_tile_url_remaining_expanded',np.concatenate((labeled[-N:], remaining)))
"""

In [58]:
"""
remaining = np.load('tile_name_tile_url_remaining_expanded.npy')
labeled = np.load("tile_name_tile_url_labeled.npy")
remaining.shape
remaining = remaining[-(137):]
labeled = np.concatenate((labeled, remaining[:(151-137)]))
#Save the ammended arrays
np.save('tile_name_tile_url_labeled',labeled)
np.save('tile_name_tile_url_remaining_expanded',remaining)
"""

(151, 2)

# FIX Crop Script

In [ ]:
from PIL import Image

def crop(path, input, height, width, k, page, area):
    im = Image.open(input)
    imgwidth, imgheight = im.size
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            box = (j, i, j+width, i+height)
            a = im.crop(box)
            try:
                o = a.crop(area)
                o.save(os.path.join(path,"PNG","%s" % page,"IMG-%s.png" % k))
            except:
                pass
            k +=1

In [ ]:
dist.tile_names = os.listdir(dist.tiles_dir) #get a list of all of the tiles in tiles directory
for tile_name in dist.tile_names: #index over the tiles in the tiles_dir 
    file_name, ext = os.path.splitext(tile_name) # File name
    print(tile_name)

    print(count)

### Pil

In [ ]:
from PIL import Image 
 
#img = Image.open(r"C:\Users\rapiduser\Downloads\shutterstock_95235265.jpg") 
img = Image.open(r"C:\chips_allocation\Check\tiles\ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif")
img.save('test.jpeg')
img = Image.open(r"C:\chips_allocation\Check\tiles\ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.")

left = 0
top = 50
right = 510
bottom = 292
 
img_res = img.crop((left, top, right, bottom)) 
 
img_res.show() 

### Open CV

In [14]:
import cv2
import os
image = cv2.imread(r"C:/chips_allocation/Check/tiles/ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif") 
y=0
x=512*19
h=512
w=512
#cropped = image[x:w, y:h]
cropped = image[y:y+h, x:x+w]
print(image.shape)

# Filename
filename = 'test2.jpg'
cv2.imwrite(os.path.join('C:/chips_allocation/Check/tiles',filename),cropped)
cv2.imshow("Cropped", cropped)
cv2.waitKey(0)
# Using cv2.imwrite() method
# Saving the image
#cv2.imwrite(filename, img)


(12220, 9770, 3)


-1

In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 999999999999
import os
file_name = "ca_060cm_2018_38121_m_3812128_ne_10_060_20180714"
#tile = Image.open(os.path.join(dist.tiles_dir, tile_name)) #open image
with Image.open(r"C:\chips_allocation\Check\tiles\ca_060cm_2018_38121_m_3812128_ne_10_060_20180714") as tile:
    width, height = tile.size
    item_width = int(512)
    col = int(width/512)+1
    row = int(height/512)+1
    count = 1
    for j in range(0,col):
        for i in range(0,row):
            box = (i*item_width, j*item_width, (i+1)*item_width, (j+1)*item_width)
            chips_save_path = os.path.join(self.chips_dir, file_name+ '_'+ \
                                           str(count).zfill(6) + '.jpg') 
            # The index is a six-digit number like '000023'. 
            #chip = tile.crop(box)
            #chip = chip.save(chips_save_path)
            chip = tile.crop(box).save(chips_save_path)
            count += 1
    print(count)

In [4]:
import numpy as np
import cv2
import os

image = cv2.imread(r"C:/chips_allocation/Check/tiles/ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif") 
y=0
x=512*19
h=512
w=512
#cropped = image[x:w, y:h]
cropped = image[y:y+h, x:x+w]

print(image.shape)

# Filename
filename = 'test2.jpg'
cv2.imwrite(os.path.join('C:/chips_allocation/Check/tiles',filename),cropped)
cv2.imshow("Cropped", cropped)
cv2.waitKey(0)
# Using cv2.imwrite() method
# Saving the image
#cv2.imwrite(filename, img)

#image = cv2.imread('C:/chips_allocation/Check/tiles/test2.jpg') 
print(chip_img.width)
print(chip_img.height)

#At the end of the image
if chip_img.width != 512:
    black_width = 512 - chip_img.width
    black_height = 512
    black_image = np.zeros((black_height,black_width,3), np.uint8)
    print("Incorrect Width")
    print(black_img.width)
    print(black_img.height)
    chip_img = np.concatenate([chip_img, blank_image])
    
if chip_img.height != 512:
    black_height = 512 - chip_img.height
    black_width = 512
    black_image = np.zeros((black_height,black_width,3), np.uint8)
    print("Incorrect Height")
    print(black_img.width)
    print(black_img.height)
    chip_img = np.concatenate([chip_img, blank_image])

In [8]:
#blank_image = np.zeros((512,512-42,3), np.uint8)
cv2.imshow('3 Channel Window', 1)
print("image shape: ", blank_image.shape)
cv2.waitKey(0)
cv2.destroyAllWindows()

image shape:  (512, 470, 3)


In [35]:
from PIL import Image


#blackBox = Image.new(im.mode, (im.width, 50), '#000')
#im.paste(blackBox, (0, im.height - blackBox.height))
#for x in range(img.width):
#    for y in range(img.height - 40, img.height):
#        img.putpixel((x, y), (0, 0, 0))

#img.save('test2_bb.png')

42
512


In [40]:
from PIL import Image  
chip_img = Image.open('C:/chips_allocation/Check/tiles/test2.jpg')
print(chip_img.width)
print(chip_img.height)

if chip_img.width != 512:
    black_width = 512 - chip_img.width
    black_height = 512
    black_img = Image.new(mode = "RGB", size = (black_width, black_height))
    print("Incorrect Width")
    print(black_img.width)
    print(black_img.height)
    chip_img+black_img
    
if chip_img.height != 512:
    black_height = 512 - chip_img.height
    black_width = 512
    blackimg = Image.new(mode = "RGB", size = (black_width, black_height))
    print("Incorrect Height")
    print(black_img.width)
    print(black_img.height)

42
512
Incorrect Width
470
512


TypeError: unsupported operand type(s) for +: 'JpegImageFile' and 'Image'

In [ ]:
import numpy as np
import cv2

image = cv2.imread('download.jpg')
y=0
x=0
h=100
w=200
crop = image[y:y+h, x:x+w]
cv2.imshow('Image', crop)
cv2.waitKey(0) 

### Wand